In [1]:
import numpy as np
import os
import pandas as pd
from tqdm.notebook import tqdm
import copy
import sys
import time

In [3]:
#Partially updated
class Graph:
    
    def __init__(self, n, mode="default", representation="matrix"):
        self.n_nodes = n
        self.mode = mode
        self.has_matrix = False
        self.matrix_error_size = None
        
        if representation == "matrix":
            try:
                self.matrix = np.full((n, n), np.inf)
                self.has_matrix = True

            except MemoryError as error:
                print("Cannot create matrix >>", error)
                ea = str(error)
                eb = ea[ea.index("iB for")-1]
                eb_m = 10**3 if eb=="G" else 10**6
                self.matrix_error_size = float(ea[ea.index("allocate")+9:ea.index("iB for")-2]) * eb_m

                self.lists = [list() for i in range(n)]
        
        elif representation == "lists":
            self.lists = [list() for i in range(n)]
        
        else:
            raise KeyError(f"Invalid representation. ({representation})")
    
    
    
    def add_edge(self, v, w, weight=1):
        
        if self.has_matrix:
            if (v != w) and (self.matrix[v-1, w-1] == np.inf):
                self.matrix[v-1, w-1] = weight
                self.matrix[w-1, v-1] = weight
        else:
            if v != w:
                if mode == "default":
                    self.lists[v-w].append(w)
                    self.lists[w-1].append(v)
                    
                if mode == "weighted":
                    if w not in np.array(graph.get_node(v))[:, 0].astype("int").tolist():
                        self.lists[v-1].append([w, weight])
                        self.lists[w-1].append([v, weight])

    
    def get_node(self, v):
        
        if not self.has_matrix:
            return self.lists[v-1]
        
        else:
            if self.mode == "default":
                return (np.where(self.matrix[v-1] == 1)[0] + 1).tolist()
            
            if self.mode == "weighted":
                neighbors = (np.where(self.matrix[v-1] < np.inf)[0] + 1).tolist()
                weights = self.matrix[v-1][self.matrix[v-1] < np.inf].tolist()
                return [list(t) for t in zip(neighbors, weights)]
    
    def get_lists(self):
        if not self.has_matrix:
            return self.lists
        else:
            return [self.get_node(n) for n in range(1, self.n_nodes + 1)]
            
    def get_matrix(self):
        if self.has_matrix:
            return self.matrix
        return None
    
    def get_matrix_beautiful(self):
        if self.has_matrix:
            return pd.DataFrame(self.matrix, columns=np.arange(1, self.n_nodes+1), index=np.arange(1, self.n_nodes+1))
        return None

In [4]:
#Already Updated
def open_graph_txt(filename, extra=False, representation="matrix"):
    with open(filename, "r") as f:
        lines = [line for line in f.read().split("\n") if line != ""]
        n_nodes = int(lines[0])
        
        if np.array([line.split(" ") for line in lines[1:]]).shape[1] < 3:
            mode = "default"
        else:
            mode = "weighted"
        
        edges = [tuple(map(lambda i: int(i), line.split(" ")[:2])) for line in lines[1:]]
        
        if mode == "default":
            weights = [1 for line in lines[1:]]
        if mode == "weighted":
            weights = [float(line.split(" ")[-1]) for line in lines[1:]]
        
        edges_weights = list(zip(edges, weights))
        
        graph = Graph(n_nodes, mode=mode, representation=representation)
        for (v, w), weight in edges_weights:
            graph.add_edge(v, w, weight=weight)
    
#     if extra:
#         return graph, n_nodes, edges

    return graph


In [5]:
graph = open_graph_txt("inputs/grafo_teste.txt", extra=False)

In [ ]:
#To Update
def graph_statistics(graph):
    print("Número de vértices:", graph.n_nodes)
    
    if graph.has_matrix:
        print("Número de arestas:", graph.get_matrix().sum()/2)
        print("Grau mínimo:", graph.get_matrix().sum(axis=0).min())
        print("Grau máximo:", graph.get_matrix().sum(axis=0).max())
        print("Grau médio:", graph.get_matrix().sum(axis=0).mean())
        print("Mediana do Grau:", np.median(graph.get_matrix().sum(axis=0)))
    else:
        print("Número de arestas:", np.sum([len(x) if i+1 not in x else len(x) + 1 for i, x in enumerate(graph.get_lists())])/2)
        print("Grau mínimo:", np.min([len(x) if i+1 not in x else len(x) + 1 for i, x in enumerate(graph.get_lists())]))
        print("Grau máximo:", np.max([len(x) if i+1 not in x else len(x) + 1 for i, x in enumerate(graph.get_lists())]))
        print("Grau médio:", np.mean([len(x) if i+1 not in x else len(x) + 1 for i, x in enumerate(graph.get_lists())]))
        print("Mediana do Grau:", np.median([len(x) if i+1 not in x else len(x) + 1 for i, x in enumerate(graph.get_lists())]))
    print("List: ", sys.getsizeof(graph.get_lists())/(10**6), "MB")
    print("Matrix: ", (str(sys.getsizeof(graph.get_matrix())/(10**6))) if graph.has_matrix else (graph.matrix_error_size), "MB")

In [ ]:
#Already updated
class DFS:
    def __init__(self, graph, root):
        self.graph = graph
        if self.graph.mode != "default":
            raise KeyError(f"Invalid mode. ({self.graph.mode})")
        
        self.visited = np.zeros(graph.n_nodes, dtype="uint8")
        self.level = np.full(graph.n_nodes, fill_value=np.inf)
        self.parent = np.full(graph.n_nodes, fill_value=-1, dtype="int32")
        self.level[root-1] = 0
        self.start_root(root)
        self.search()
    
    def start_root(self, root):
        self.stack = []
        self.stack.append(root)
    
    def search(self):
        while(len(self.stack) != 0):
            u = self.stack.pop()
            
            if not self.visited[u-1]:
                self.visited[u-1] = 1
                
                for v in sorted(self.graph.get_lists()[u-1], reverse=True):
                    if not self.visited[v-1]:
                        self.stack.append(v)
                        self.parent[v-1] = u
                        self.level[v-1] = self.level[u-1] + 1

In [6]:
#Already updated
class BFS:
    def __init__(self, graph, root):
        self.graph = graph
        if self.graph.mode != "default":
            raise KeyError(f"Invalid mode. ({self.graph.mode})")
        
        self.visited = np.zeros(graph.n_nodes, dtype="uint8")
        self.level = np.full(graph.n_nodes, fill_value=np.inf)
        self.parent = np.full(graph.n_nodes, fill_value=-1, dtype="int32")
        
        self.level[root-1] = 0
        self.visited[root-1] = 1
        
        self.start_root(root)
        self.search()
        
    def start_root(self, root):
        self.queue = []
        self.queue.append(root)
        
    def search(self):
        
        while(len(self.queue)):
            v = self.queue.pop(0)
            
            for w in sorted(self.graph.get_lists()[v-1]):
                if v == w:
                        continue
                if not self.visited[w-1]:
                    self.visited[w-1] = 1
                    self.queue.append(w)
                    self.parent[w-1] = v
                    self.level[w-1] = self.level[v-1] + 1

In [ ]:
#To Update

class MinimumPath:
    
    def __init__(self, graph):
        self.graph = graph
        self.matrix = np.full((graph.n_nodes, graph.n_nodes), fill_value=-1, dtype="int32")
        self.run()
    
    def run(self):
        for v in tqdm(range(1, self.graph.n_nodes+1)):
            bfsl = BFSL(self.graph, v)
            bfsl.search()
            for bfsl_node_index in np.argwhere(bfsl.visited == 1).reshape(-1):
                self.matrix[v-1, bfsl_node_index] = bfsl.level[bfsl_node_index]
            del bfsl
    
    def get_distance(self, u, v):
        return self.matrix[u-1, v-1]
    
    def get_diameter(self):
        return np.max(self.matrix)
    
    def get_matrix(self):
        return self.matrix
    
    def get_matrix_beautiful(self):
        return pd.DataFrame(self.matrix, columns=np.arange(1, self.graph.n_nodes+1), index=np.arange(1, self.graph.n_nodes+1))
    

In [ ]:
#To Update
class Components:
    
    def __init__(self, graph):
        self.graph = graph
        self.visited = np.zeros(graph.n_nodes, dtype="uint8")
        self.components = []
        
        while np.argwhere(self.visited == 0).reshape(-1).shape[0] > 0:
            root = np.argwhere(self.visited == 0).reshape(-1)[0] + 1

            bfsl = BFSL(self.graph, root)
            bfsl.search()
            
            bfsl_visited_index = np.argwhere(bfsl.visited == 1).reshape(-1)
            
            self.visited[bfsl_visited_index] = 1
            self.components.append((bfsl_visited_index+1).tolist())

    def get_components(self):
        a = sorted(self.components, key=lambda x: len(x), reverse=True)
        b = [len(x) for x in a]
        c = list(zip(b, a))
        return c


In [ ]:
# from pyvis.network import Network

# net = Network(notebook=True)

# for v in range(1, graph.n_nodes+1):
#     net.add_node(v, label=v)
    
# for v, neighbors in enumerate(graph.nodes):
#     for w in neighbors:
#         net.add_edge(v+1, w)

# net.show("graph.html")

In [ ]:
# folder = "inputs"
# filename = "grafo_5.txt"

# path = os.path.join(folder, filename)

# graph = open_graph_txt(path)
# graph.sort_neighbors()

# # print(graph.has_matrix)
# # graph_statistics(graph)
# # graph.has_matrix = False
# # print("----")
# # print(graph.has_matrix)
# graph_statistics(graph)

In [ ]:
dfs = DFS(graph, 1)
dfs.search()

print("DFS Result:")
dfs_df = pd.DataFrame(list(zip(range(1, dfs.graph.n_nodes+1), dfs.level, dfs.parent)), columns=["node", "level", "parent"], index=np.arange(1, dfs.graph.n_nodes+1))
# dfs_df.to_csv("./outputs/dfs_test.csv")
dfs_df

In [ ]:
bfs = BFS(graph, 1)
bfs.search()

print("BFS Result:")
bfs_df = pd.DataFrame(list(zip(range(1, bfs.graph.n_nodes+1), bfs.level, bfs.parent)), columns=["node", "level", "parent"], index=np.arange(1, bfs.graph.n_nodes+1))
# bfs_df.to_csv("./outputs/bffs_test.csv")
bfs_df

In [ ]:
########################################

In [ ]:
minpath = MinimumPath(graph)

print("Minpath Distance:", minpath.get_distance(1, 3))
print("Minpath Diameter:", minpath.get_diameter())

In [ ]:
components = Components(graph)

with open("./outputs/components_test.txt", mode="w") as out:
    for line in components.get_components():
        out.write(str(line))
        out.write("\n")

In [ ]:
########################################

In [10]:
class Dijkstra:
    def __init__(self, graph, root):
        if graph.mode == "default":
            self.graph = graph
            self.bfs = BFS(graph, root)
            self.distance = self.bfs.level
            self.parent = self.bfs.parent

        elif graph.mode == "weighted":
            self.graph = graph
            
            self.distance = np.full(graph.n_nodes, fill_value=np.inf)
            self.explored = np.zeros(graph.n_nodes, dtype="uint8")
            self.parent = np.full(graph.n_nodes, fill_value=-1, dtype="int32")
            self.found = {root}

            self.distance[root-1] = 0
            self.parent[root-1] = -1
            
            self.find()
        
        else:
            raise KeyError(f"Invalid graph mode. ({graph.mode})")
    
    def find(self):
        while len(self.found) != 0:
            u = np.argmin(np.where(self.explored == 0, self.distance, np.inf)) + 1
            self.found.remove(u)
            self.explored[u-1] = 1
            for v, weight in self.graph.get_node(u):
                if weight < 0: raise ValueError(f"Weight cannot be negative. (weight = {weight})")
                if self.explored[v-1] == 1: continue
                self.found.add(v)
                if self.distance[v-1] > weight + self.distance[u-1]:
                    self.distance[v-1] = weight + self.distance[u-1]
                    self.parent[v-1] = u

In [11]:
dijkstra = Dijkstra(graph, 1)
print("Dijkstra Result:")
dijkstra_df = pd.DataFrame(list(zip(range(1, dijkstra.graph.n_nodes+1), dijkstra.distance, dijkstra.parent)), columns=["node", "distance", "parent"], index=np.arange(1, dijkstra.graph.n_nodes+1))
dijkstra_df

Dijkstra Result:


,node,distance,parent
1,1,0.0,-1
2,2,1.0,1
3,3,1.0,1
4,4,1.0,1
5,5,2.0,4


In [ ]:
dijkstra_df[dijkstra_df["parent"] == -1]